Find how many unique movies (movieId) this dataset does contain
Find the top five users (userId) who put in the most rankings
Filter out rankings in between stars (only 1, 2, 3, 4, 5) and calculate the number of ratings per category (Hint: Use RDD.filter() and modulo 1)
Load movies.csv and show the titles of the top 5 most rated movies 

In [9]:
# Import the findspark module 
import findspark

# Initialize via the full spark path
findspark.init("/usr/local/spark/")
#findspark.init("C:/spark/")

In [10]:
# Import the SparkSession module
from pyspark.sql import SparkSession

# Import the collections module
import collections

# Gets an existing :class:`SparkSession` or, if there is no existing one, creates a
# new one based on the options set in this builder.
spark = SparkSession.builder \
   .master("local[8]") \
   .appName("RatingsHistogram") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()

# Main entry point for Spark functionality. A SparkContext represents the
# connection to a Spark cluster, and can be used to create :class:`RDD` and
# broadcast variables on that cluster.
sc = spark.sparkContext

In [13]:
movies = sc.textFile("./data/movies.csv", minPartitions=4)
ratings = sc.textFile("./data/ratings.csv", minPartitions=4)


header = movies.first()
movies = movies.filter(lambda row: row != header)

header = ratings.first()
ratings = ratings.filter(lambda row: row != header)

Find how many unique movies (movieId) this dataset does contain

In [53]:
uniqueMovies = ratings.map(lambda x: (x.split(',')[1]))
uniqueMovies.count()

100836

Find the top five users (userId) who put in the most rankings

In [54]:
allUsers = ratings.map(lambda x: (x.split(',')[0])).countByValue()
topUsers = sorted(allUsers.items(), key=lambda x: x[1], reverse=True)
top5Users = topUsers[:5]
top5Users

[('414', 2698), ('599', 2478), ('474', 2108), ('448', 1864), ('274', 1346)]

Filter out rankings in between stars (only 1, 2, 3, 4, 5) and calculate the number of ratings per category (Hint: Use RDD.filter() and modulo 1)

In [55]:
allRankings = ratings.map(lambda x: x.split(',')[2])
allRankings.take(15)
def filterfun(row):
    rating = row[0]
    try:
        rating = float(rating)
    except:
        print('invalid rating submitted')
        
    return rating % 1 == 0

filteredRankings = allRankings.filter(filterfun)
filteredRankings.take(15)

['4.0',
 '4.0',
 '4.0',
 '5.0',
 '5.0',
 '3.0',
 '5.0',
 '4.0',
 '5.0',
 '5.0',
 '5.0',
 '5.0',
 '3.0',
 '5.0',
 '4.0']

Load movies.csv and show the titles of the top 5 most rated movies 

In [56]:
rankedMovies = ratings.map(lambda x: (x.split(',')[1], 1)).reduceByKey(lambda a, b: a + b)
allMovies = movies.map(lambda x: (x.split(',')[0], x.split(',')[1]))
joined = rankedMovies.join(allMovies)
cleaned = joined.map(lambda x: (x[1][1], x[1][0]))
top5RatedMovies = cleaned.sortBy(lambda x: x[1], ascending=False).take(5)
top5RatedMovies

[('Forrest Gump (1994)', 329),
 ('"Shawshank Redemption', 317),
 ('Pulp Fiction (1994)', 307),
 ('"Silence of the Lambs', 279),
 ('"Matrix', 278)]